# FERNANDO LEON FRANCO | PRACTICA MODELO DE LENGUAJE PROBABILISTA

In [47]:
import math
from itertools import permutations
import nltk
from nltk.tokenize import TweetTokenizer
from time import sleep
import random
import sys

In [48]:
def get_texts_from_file(path_corpus, path_truth):

    tr_txt = [] # Aqui van los twits
    tr_y = [] # Aqui van las etiquetas

    with open(path_corpus, 'r', encoding='utf-8') as f_corpus, open(path_truth , 'r', encoding='utf-8') as f_truth:
        
        for twitt in f_corpus:
            tr_txt += [twitt]
            
        for label in f_truth:
            tr_y += [label]
    
    return tr_txt,tr_y        

In [49]:


path_global = "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/mex"
path_corpus = path_global + '/mex20_train.txt'
path_truth = path_global + '/mex20_train_labels.txt'


tr_txt, tr_y = get_texts_from_file(path_corpus,path_truth)



# Construir los datos de validación
path_corpus = path_global + '/mex20_val.txt'
path_truth = path_global + '/mex20_val_labels.txt'


va_txt, va_y = get_texts_from_file(path_corpus,path_truth)


# PARTE 1: Procesamiento y tratamiento de los datos para un modelo de lenguaje probabilista
- Necesitamos contar las ocurrencias de trigramas o de bigramas en el conjunto de datos de entrenamiento.

In [50]:
class TrigramData:
    MODE = "TRIGRAM"
    def __init__(self, vocab_max, tokenizer):
        self.vocab_max = vocab_max
        self.tokenizer = tokenizer
        self.final_vocab = set()
        self.SOS = "<s>"
        self.EOS = "</s>"
        self.UNK = "<unk>"
        
    def fit(self, raw_text): # En row_text recibiré los tweets
        self.freq_dist = nltk.FreqDist()
        tokenized_corpus = []
        
        for txt in raw_text:
            tokens = self.tokenizer.tokenize(txt.lower())
            tokenized_corpus.append(tokens) # recordar que esta es una lista de listas de tweets tokenizados
            for w in tokens:
                self.freq_dist[w] += 1

        self.final_vocab = { token for token, _ in self.freq_dist.most_common(self.vocab_max) }
        self.final_vocab.update([self.SOS, self.EOS, self.UNK])
        
        transform_corpus = []
        for tokens in tokenized_corpus:
            transform_corpus.append(self.transform(tokens)) # tokens es un tweet tokenizado
        
        
        return transform_corpus
    
    
    def mask_out_of_vocab(self, word):
        if word in self.final_vocab:
            return word
        else:
            return self.UNK

    def add_sos_eos(self, tokenized_text):
        
        if self.MODE == "BIGRAM":
            return [self.SOS] + tokenized_text + [self.EOS]
        elif self.MODE == "TRIGRAM":
            return [self.SOS, self.SOS] + tokenized_text + [self.EOS]

    def transform(self, tokenized_text):
        transformed = [] # Tokens transformados
        for w in tokenized_text:
            transformed.append(self.mask_out_of_vocab(w)) # Mask  Out of Vocabulary Word
        transformed = self.add_sos_eos(transformed)

        return transformed

In [70]:
TOP_PALABRAS = 10_000
tokenizador = TweetTokenizer()

trigram_data = TrigramData(vocab_max=TOP_PALABRAS, tokenizer=tokenizador)

transformed_corpus = trigram_data.fit(tr_txt)

print(transformed_corpus)

[['<s>', '<s>', '@usuario', '@usuario', '@usuario', 'q', 'se', 'puede', 'esperar', 'del', 'maricon', 'de', 'closet', 'de', 'la', 'yañez', 'aun', 'recuerdo', 'esa', 'ves', 'q', 'lo', 'vi', 'en', 'zona', 'rosa', 'viendo', 'quien', 'lo', 'levantada', '</s>'], ['<s>', '<s>', '@usuario', 'la', 'piel', 'nueva', 'siempre', 'arde', 'un', 'poquito', 'los', 'primeros', 'días', '...', 'y', 'más', 'con', 'este', 'puto', 'clima', '</s>'], ['<s>', '<s>', 'ustedes', 'no', 'se', 'enamoran', 'de', 'mí', '…', 'por', 'tontas', '.', '</s>'], ['<s>', '<s>', 'me', 'las', 'va', 'a', 'pagar', 'esa', 'puta', 'gorda', 'roba', 'tuits', '...', '</s>'], ['<s>', '<s>', '@usuario', 'la', 'gente', 'es', 'tonta', 'porque', 'no', 'se', 'dan', 'cuenta', 'que', 'tú', 'haces', 'a', 'batman', 'azul', '</s>'], ['<s>', '<s>', 'estoy', 'muy', 'encabronada', 'con', 'las', 'pseudo', 'feministas', 'por', 'tontas', 'e', 'iletradas', ',', 'a', 'veces', 'me', 'avergüenza', 'ser', 'mujer', ';', 'preferiría', 'tener', 'un', 'falo', '

In [52]:
final_vocab = trigram_data.final_vocab
print(f"Tamaño del vocabulario final: {len(final_vocab):,}")

Tamaño del vocabulario final: 10,003


In [53]:
for palabra in list(final_vocab)[:20]:
    print(palabra)

ok
adeco
bloquee
futbol
😔
márquez
harán
propios
vien
💖
martinolli
elegir
cdtm
primer
sur
preocupate
clavar
ofensiva
usuario
riquísima


In [54]:
print("Primeras 20 palabras del vocabulario final:")
for idx, palabra in enumerate(sorted(final_vocab)):
    if idx < 20:
        print(f"{idx+1:2d}. {palabra}")




Primeras 20 palabras del vocabulario final:
 1. !
 2. "
 3. #
 4. #140caracteres
 5. #19s
 6. #280caracteres
 7. #aba
 8. #aborto
 9. #acropolispuebla
10. #addi
11. #adn
12. #agsmx
13. #aguascalientes
14. #aguilas
15. #ahscult
16. #alaorden
17. #alerta
18. #alvlavida
19. #amedroauditor
20. #amigos


# BUILDING A TRIGRAM LANGUAGE MODEL

In [55]:
class TrigramLanguageModel:
    """ Modelo interpolado  unigramas + bigramas + trigramas """
    def __init__(self, lambda_1 = 0.59, lambda_2 = 0.40, lambda_3 = 0.01, vocab=None):
        # Las lambdas deben sumar 1 y son los pesos de cada modelo
        self.lambda_1 = lambda_1 # Trigramas
        self.lambda_2 = lambda_2 # Bigramas
        self.lambda_3 = lambda_3 # Unigramas
        
        # Contadores
        self.unigram_counts = {} # Los unigramas con las palabras solitas
        self.bigram_counts = {}  # Los bigramas subsecuencias de tamaño 2
        self.trigram_counts = {} # Los trigramas subsecuencias de tamaño 3

        self.vocab = vocab
        self.vocab_size = len(vocab) if vocab is not None else 0
        self.SOS = "<s>"
        self.EOS = "</s>"
        self.UNK = "<unk>"


    def train(self, transformed_corpus):
        for tokens in transformed_corpus: # primero recorro tweet por tweet
            for i, word in enumerate(tokens): # Luego para cada tweet le reccorro sus palabras
                
                # Unigramas
                self.unigram_counts[word] = self.unigram_counts.get(word, 0) + 1
                
                # Bigramas
                if i > 0: # Solo si ya vi un palabra antes, puedo formar un bigrama
                    bigrama = (tokens[i-1], word)
                    self.bigram_counts[bigrama] = self.bigram_counts.get(bigrama, 0) + 1
                
                # Trigramas
                if i > 1: # Solo si ya vi dos palabras antes, puedo formar un trigrama
                    trigrama = (tokens[i-2], tokens[i-1], word)
                    self.trigram_counts[trigrama] = self.trigram_counts.get(trigrama, 0) + 1
            
            self.total_tokens = sum(self.unigram_counts.values())
    
    
    def mask_out_of_vocab(self, word):
        return "<unk>" if word not in self.vocab else word
    
    
    def unigram_probability(self, word):
        numerador = self.unigram_counts.get(self.mask_out_of_vocab(word), 0) + 1
        denominador = self.total_tokens + self.vocab_size
        return numerador / denominador
                    
                    
    def bigram_probability(self, word_prev, word): # Esta función calcula P(word | word_prev)
        w_prev = self.mask_out_of_vocab(word_prev)
        w = self.mask_out_of_vocab(word)
        bigrama = (w_prev, w) # P(w | w_prev)
        
        numerador = self.bigram_counts.get(bigrama, 0) + 1
        denominador = self.unigram_counts.get(w_prev, 0) + self.vocab_size
        return  numerador / denominador
    
    
    def trigram_probability(self, word_prev2, word_prev1, word): # Esta función calcula P(word | word_prev2, word_prev1)
        w_prev2 = self.mask_out_of_vocab(word_prev2)
        w_prev1 = self.mask_out_of_vocab(word_prev1)
        w = self.mask_out_of_vocab(word)
        
        trigrama = (w_prev2, w_prev1, w)
        bigrama = (w_prev2, w_prev1)
        numerador = self.trigram_counts.get(trigrama, 0) + 1
        denominador = self.bigram_counts.get(bigrama, 0) + self.vocab_size
        return  numerador / denominador
        
    def check_probs(self):
        print(sum(self.unigram_probability(w) for w in self.vocab)) # Type: ignore
        print(sum(self.bigram_probability("gato", w) for w in self.vocab)) # Type: ignore
        print(sum(self.trigram_probability("hola", "como", w) for w in self.vocab)) # Type: ignore

    def probabilidades_uso(self, word_prev2, word_prev1, word):
        p_trigrama = self.trigram_probability(word_prev2, word_prev1, word)
        p_bigrama = self.bigram_probability(word_prev1, word)
        p_unigrama = self.unigram_probability(word)

        return p_trigrama,p_bigrama,p_unigrama

    def probability_of_word(self, word_prev2, word_prev1, word):
        p_trigrama,p_bigrama,p_unigrama = self.probabilidades_uso(word_prev2, word_prev1, word)

        return (self.lambda_1 * p_trigrama) + (self.lambda_2 * p_bigrama) + (self.lambda_3 * p_unigrama)


    def top_next_words(self, w_prev2, w_prev, top_k=5):
        candidates = []
        for cand in self.vocab:
            p_w = self.probability_of_word(w_prev2, w_prev, cand)
            candidates.append((cand, p_w))
        # Ordena de mayor a menor probabilidad
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[:top_k]
        
    
    def secuence_probability(self, seciencia_tokens):
        probabilidad_acumulada = 0
        for i in range(2, len(seciencia_tokens)):
            w_prev2 = seciencia_tokens[i-2]
            w_prev1 = seciencia_tokens[i-1]
            w = seciencia_tokens[i]
            probabilidad = self.probability_of_word(w_prev2, w_prev1, w)
            probabilidad_acumulada += math.log(probabilidad)

        return math.exp(probabilidad_acumulada)

    def rank_permutations(self, tokens, top_k=5, bottom_k=5):
        permutaciones = list(permutations(tokens))
        
        scores = []
        for perm in permutaciones:
            p_sencuencia = self.secuence_probability(perm)
            scores.append((" ".join(perm), p_sencuencia))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        best = scores[:top_k]
        worst = scores[-bottom_k:]
        
        return best,worst
    
    
    def generate_from_tokens_as_chatgpt(self, first_word, second_word, max_length=20, delay=0.5):
        def validar_palabra(word):
            return word if word in self.vocab else "<unk>"
        
        first_word = validar_palabra(first_word)
        second_word = validar_palabra(second_word)
        
        generated_tokens = [first_word, second_word]
        
        print(first_word, second_word, end='', flush=True)
        
        for _ in range(max_length):
            w_prev2 = generated_tokens[-2]
            w_prev1 = generated_tokens[-1]
            
            distribucion = []
            total_p =  0.0
            for cand in self.vocab: # type: ignore
                p_w = self.probability_of_word(w_prev2, w_prev1, cand)
                distribucion.append((cand, p_w))
                total_p += p_w
            
            if total_p == 0:
                break
            
            r = random.random() * total_p
            acumulado = 0.0
            siguiente_palabra = None
            
            for w, p in distribucion:
                acumulado += p
                if acumulado >= r:
                    siguiente_palabra = w
                    break
                
            if siguiente_palabra == self.EOS or siguiente_palabra is None:
                break
            generated_tokens.append(siguiente_palabra)
            print(" " + siguiente_palabra, end='', flush=True)
            sleep(delay)
        print()
        return generated_tokens
            
        
        

In [56]:
# EJEMPLO DE LAS PERMUTACIONES
test_tokens = ["sino","gano","me"]
permitaciones = set(permutations(test_tokens))
print(f"Tamaño de las permutaciones: {len(permitaciones)}")
for p in list(permitaciones)[:10]:
    print(p)

Tamaño de las permutaciones: 6
('sino', 'gano', 'me')
('gano', 'me', 'sino')
('sino', 'me', 'gano')
('me', 'sino', 'gano')
('gano', 'sino', 'me')
('me', 'gano', 'sino')


In [57]:
final_vocab = trigram_data.final_vocab

trigram_lm = TrigramLanguageModel(vocab=final_vocab)
trigram_lm.train(transformed_corpus)

In [58]:
trigram_lm.check_probs()

1.0
1.0
1.0


# PRUEBAS

In [59]:
word_prev2, word_prev1, word = "<s>", "hola", "mundo"
p_w = trigram_lm.probability_of_word(word_prev2, word_prev1, word)
print(f"P({word} | {word_prev2}, {word_prev1}) = {p_w}")

P(mundo | <s>, hola) = 0.00010335132933804556


In [60]:
word_prev2, word_prev1, word = "<s>", "saludos", "a"
p_w = trigram_lm.probability_of_word(word_prev2, word_prev1, word)
print(f"P({word} | {word_prev2}, {word_prev1}) = {p_w}")

P(a | <s>, saludos) = 0.0002778290833875056


In [61]:
word_prev2, word_prev1, word = "hijo", "de", "tu"
p_w = trigram_lm.probability_of_word(word_prev2, word_prev1, word)
print(f"P({word} | {word_prev2}, {word_prev1}) = {p_w}")

P(tu | hijo, de) = 0.0034786898082040775


In [62]:
word_prev2, word_prev1, word = "vete", "a", "la"
p_w = trigram_lm.probability_of_word(word_prev2, word_prev1, word)
print(f"P({word} | {word_prev2}, {word_prev1}) = {p_w}")

P(la | vete, a) = 0.011153217757174417


In [63]:
top_5 = trigram_lm.top_next_words("hijo", "de", top_k=5)
for w, p in top_5:
    print(f"P({w} | hijo, de) = {p}")

P(la | hijo, de) = 0.009244873899638673
P(<unk> | hijo, de) = 0.005531514118896191
P(tu | hijo, de) = 0.0034786898082040775
P(su | hijo, de) = 0.0026121131631923304
P(mi | hijo, de) = 0.0025834426767438


In [64]:
top_5 = trigram_lm.top_next_words("hola", "mundo", top_k=5)
for w, p in top_5:
    print(f"P({w} | hola, mundo) = {p}")

P(<s> | hola, mundo) = 0.0009548905419916221
P(. | hola, mundo) = 0.0007077447034876222
P(</s> | hola, mundo) = 0.0006859567869160297
P(, | hola, mundo) = 0.0005422899688591583
P(de | hola, mundo) = 0.00042929561320088607


In [65]:
top_5 = trigram_lm.top_next_words("saludos", "desde", top_k=5)
for w, p in top_5:
    print(f"P({w} | saludos, desde) = {p}")

P(<s> | saludos, desde) = 0.0009548787514258432
P(que | saludos, desde) = 0.0007886220919455008
P(el | saludos, desde) = 0.0006111244424612674
P(</s> | saludos, desde) = 0.000526851827413192
P(la | saludos, desde) = 0.00044192585759427003


In [66]:
top_5 = trigram_lm.top_next_words("partido", "de", top_k=5)
for w, p in top_5:
    print(f"P({w} | partido, de) = {p}")

P(la | partido, de) = 0.00906949641563996
P(<unk> | partido, de) = 0.005473304741341722
P(mi | partido, de) = 0.002583817352411454
P(los | partido, de) = 0.002471707927558721
P(que | partido, de) = 0.0023270595921347985


In [67]:
top_5 = trigram_lm.top_next_words("como", "estas", top_k=5)
for w, p in top_5:
    print(f"P({w} | como, estas) = {p}")

P(<s> | como, estas) = 0.0009549202710826276
P(</s> | como, estas) = 0.0006256786737036542
P(<unk> | como, estas) = 0.00035164397556725974
P(que | como, estas) = 0.00035115739702537346
P(de | como, estas) = 0.00034977875782336225


# EVALUACION CUALITATUVA CON LOS MODELOS DE LENGUAJE

In [68]:
test_tokens = ["sino",
               "gano",
               #",",
               "me",
               "voy",
               "a",
               "la",
               "chingada",
            #    "ahora",
            #    "si"

               ]

precision_float = 17
probabilidad_sencuencia = trigram_lm.secuence_probability(test_tokens)
print(f'Probabilidad de la secuencia "{' '.join(test_tokens)}": {probabilidad_sencuencia:.{precision_float}f} [{precision_float}]')



top_5, bottom_5 = trigram_lm.rank_permutations(test_tokens, top_k=5, bottom_k=5)
print("\nTop 5 permutaciones:")
for seq, p in top_5:
    print(f'Probabilidad de:"{seq}") = {p:.{precision_float}f} [{precision_float}]')
    
precision_float = 20
print("\nBottom 5 permutaciones:")
for seq, p in bottom_5:
    print(f'Probabilidad de:"{seq}") = {p:.{precision_float}f} [{precision_float}]')

Probabilidad de la secuencia "sino gano me voy a la chingada": 0.00000000000002799 [17]

Top 5 permutaciones:
Probabilidad de:"sino gano me voy a la chingada") = 0.00000000000002799 [17]
Probabilidad de:"gano sino me voy a la chingada") = 0.00000000000002799 [17]
Probabilidad de:"gano me voy a la chingada sino") = 0.00000000000001271 [17]
Probabilidad de:"sino me voy a la chingada gano") = 0.00000000000001263 [17]
Probabilidad de:"sino gano voy a la chingada me") = 0.00000000000000241 [17]

Bottom 5 permutaciones:
Probabilidad de:"a la sino chingada voy me gano") = 0.00000000000000000002 [20]
Probabilidad de:"a la gano chingada voy me sino") = 0.00000000000000000002 [20]
Probabilidad de:"a la gano me sino voy chingada") = 0.00000000000000000002 [20]
Probabilidad de:"a la sino voy chingada me gano") = 0.00000000000000000002 [20]
Probabilidad de:"a la gano voy chingada me sino") = 0.00000000000000000002 [20]


In [69]:
respuesta = trigram_lm.generate_from_tokens_as_chatgpt("puta", "madre", max_length=100, delay=0.3)



puta madre demuestra veías adivinos duro hetero comfort dándole word 😚 pregunto súbele manta cerca alrrevez redactan quinta mitoteras informarse volvieron solares

KeyboardInterrupt: 